# Абстрактивная суммаризация

### Контекст

Необходимо провести суммаризацию полученной информации методами абстрактивной сумммаризации.

Абстрактивная суммаризация – это метод автоматического реферирования текста, при котором создается новое краткое изложение с использованием перефразирования и генерации новых предложений, а не просто извлечения фрагментов из исходного текста.

### Что нужно сделать?

Попробовать разыне методы абстрактивной суммаризации:

- T5 (Text-to-Text Transfer Transformer) - https://github.com/google-research/text-to-text-transfer-transformer
- BART https://huggingface.co/docs/transformers/model_doc/bart

### Результат:

- код .py, .ipynb
- выводы

### Загрузка библиотек и блок используемых функций

In [1]:
# Импорт библиотек
import os
import pandas as pd
import numpy as np

!pip install sentencepiece --quiet
!pip install protobuf --quiet
!pip install --upgrade protobuf
from transformers import (
    MT5ForConditionalGeneration, 
    T5Tokenizer, 
    MBartForConditionalGeneration, 
    MBart50TokenizerFast,
    AutoTokenizer,
    AutoModelForSeq2SeqLM, 
    AutoModelForCausalLM)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Загрузка csv полученного из парсинга json и удаленеие строк, в которых столбец text пустой

def load_df(file, index=None):
    pth1 = os.path.join('data', 'example', file)
    pth2 = file

    if os.path.exists(pth1):
        df = pd.read_csv(pth1, na_values=np.nan)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2, na_values=np.nan)
    else:
        print('Что-то пошло не так')
        return None
        
    try:
        df = df.dropna(subset=['text'])
    except:
        print('Столбец "text" отсутствует в датафрейме')
        
    return df

In [3]:
# Разобьем сообщения на части (чанки) по заданному количеству слов

def split_on_chunks(df, max_words_per_chunk=500):
    current_words = 0
    current_chunk = []
    chunks = []
    
    for index, row in df.iterrows():
        # Считаем количество слов в текущем сообщении
        words_in_message = len(row['text'].split())
        
        # Если сообщение превышает лимит, разбиение на несколько частей
        if words_in_message > max_words_per_chunk:
            # Разбиваем длинное сообщение на несколько частей
            words = row['text'].split()
            while len(words) > max_words_per_chunk:
                part = ' '.join(words[:max_words_per_chunk])
                chunks.append([part])  # Добавляем эту часть как отдельный чанк
                words = words[max_words_per_chunk:]
            # Оставшуюся часть добавляем как последний чанк
            if words:
                chunks.append([' '.join(words)])
        elif current_words + words_in_message > max_words_per_chunk:
            # Если добавление сообщения превысит лимит, добавляем текущий чанк
            chunks.append(current_chunk)
            # Начинаем новый чанк с текущего сообщения
            current_chunk = [row['text']]
            current_words = words_in_message
        else:
            # Добавляем сообщение к текущему чанку
            current_chunk.append(row['text'])
            current_words += words_in_message
    
    # Добавляем последний чанк, если он есть
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

In [4]:
# Функция суммаризации текста
"""
def summarize_chunk(chunk, tokenizer, model):
    # Объединяем сообщения части в один текст
    text = ' '.join(chunk)
    
    # Форматируем для модели
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Генерируем резюме
    summary_ids = model.generate(inputs, num_beams=3, max_length=150, min_length=50, length_penalty=1.5, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
"""

'\ndef summarize_chunk(chunk, tokenizer, model):\n    # Объединяем сообщения части в один текст\n    text = \' \'.join(chunk)\n    \n    # Форматируем для модели\n    input_text = "summarize: " + text\n    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n    \n    # Генерируем резюме\n    summary_ids = model.generate(inputs, num_beams=3, max_length=150, min_length=50, length_penalty=1.5, early_stopping=True)\n    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)\n'

In [5]:
# Общая функция суммаризации текста
def summarize_chunk(chunk, tokenizer, model, prefix=''):
    # Объединяем сообщения части в один текст
    text = ' '.join(chunk)
    
    # Форматируем для модели
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="longest")
    
    # Генерируем резюме
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, min_length=50, length_penalty=2, early_stopping=True)
    
    # Декодируем результат
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summary = summary.replace("summarize: ", "") # убираем "summarize: " для BART
    return summary

In [6]:
# Отдельная функция для ruGPT3
def summarize_with_rugpt3(text, tokenizer, model, max_length=150):
    prompt = f"Текст: {text.strip()}\nКраткое содержание:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)

    output_ids = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_length,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        top_k=50,
        num_beams=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    summary_start = generated.find("Краткое содержание:")
    return generated[summary_start + len("Краткое содержание:"):].strip() if summary_start != -1 else generated


### Загрузка данных

In [7]:
df = load_df('data.csv')
df.head()

,date,chat_name,chat_id,sender_id,username,text
2,2025-02-03T11:28:38,💬 Data Practicum Chat,1379846874,user312724902,Olga Varavina,Всем большой привет! Приглашаю на свой уютный ...
3,2025-02-03T11:52:20,💬 Data Practicum Chat,1379846874,user1349934990,Илья,А у тебя когда будет свой канал про аналитику?
4,2025-02-03T11:52:37,💬 Data Practicum Chat,1379846874,user1349934990,Илья,Будешь туда голосовухи пятиминутные постить
5,2025-02-03T11:55:09,💬 Data Practicum Chat,1379846874,user60031833,Sergey,"Потому что сделаны так, будто устарели уже лет..."
6,2025-02-03T11:56:57,💬 Data Practicum Chat,1379846874,user60031833,Sergey,Подкаст?)


In [8]:
chunks = split_on_chunks(df)

### Загрузка моделей

In [9]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google/mt5-small", legacy=False)
model_t5 = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [10]:
tokenizer_bart = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="ru_RU", tgt_lang="ru_RU")
model_bart = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

In [11]:
tokenizer_rut5 = AutoTokenizer.from_pretrained("cointegrated/rut5-small")
model_rut5 = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
tokenizer_gpt3 = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model_gpt3 = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

pytorch_model.bin:  16%|#5        | 493M/3.14G [00:00<?, ?B/s]

### Суммаризация текста

In [13]:
chunk_summaries_t5 = [summarize_chunk(chunk, tokenizer_t5, model_t5, prefix="summarize: ") for chunk in chunks]

In [14]:
chunk_summaries_bart = [summarize_chunk(chunk, tokenizer_bart, model_bart) for chunk in chunks]

In [15]:
chunk_summaries_rut5 = [summarize_chunk(chunk, tokenizer_rut5, model_rut5, prefix="summarize: ") for chunk in chunks]

In [16]:
chunk_summaries_gpt3 = [summarize_with_rugpt3(' '.join(chunk), tokenizer_gpt3, model_gpt3) for chunk in chunks]

In [17]:
chunk_summaries_t5

['<extra_id_0> спасибо))))))))))))))))))))))))))))))))))))))))))))))))))))))))))):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)):)',
 '<extra_id_0> и т.д.))))))))))))))))))))))))))))))))))))))))))))))))))) ().) )) )) )) )) )) )) )) ))) ))) )) ))) ))) ))) ))) ))))))))))))))))))))))))))))))))))))))',
 '<extra_id_0> не знаю. Интересно, а написать в сопровождение. Инсайты. Можно в любом формате. Может. можно только в любом формате. Может. можно.',
 '<extra_id_0>? Ответ очевиден. Ответ очевиден. Ответ очевиден. Ответ очевиден. Ответы не сбылись. Ответы не сбылись. Ответы не сбылись.',
 '<extra_id_0> спасибо)))))))))))))))))))))))))))))))))))))))))))))))))',
 '<extra_id_0> ссылка неактивна и все тут уже нет вопросов))))))))))))))))))))))))))))))))))))))))))))))))))))))',
 '<extra_id_0> Спасибо Ире))))))))))))))))))))))))))))))))))))))))) )))))))))',
 '<extra_id_0> спасибо) Спасибо)))))))))))))))))))))))))))))))

In [18]:
chunk_summaries_bart

['Всем большой привет! Приглашаю на свой уютный канал Диванные данные. Присаживайтесь поудобнее, обсудим вкатывание (или скатывание) в аналитику, прохождение собесов и многое другое. https://t.me/Divan_data А у тебя когда будет свой канал про аналитику? Будешь туда голосовухи пятиминутные постить Потому что сделаны так, будто устарели уже лет на цать, особенно кажется так по поводу ошибки с дизайном времен xp. Но новый логотип мне все равно нравится) Подкаст?) Не, это не так раздражает Нужны голосовуки с э',
 'Предложение поучаствовать в бета-тестировании Продуктового аналитика и BI-аналитика *для выпускников курсов "Аналитик данных", "Аналитик данных расширенный" и "Аналитик данных Bootcamp" Всем привет! На связи команда курсов Аналитики данных. Мы активно работаем над курсами Продуктового аналитика и BI-аналитика, и сейчас мы в поиске бета-тестировщиков новых курсов. Для тестирования необходимы навыки в аналитике данных, именно поэтому мы пришли к вам — нашим выпускникам #бета-тест —

In [19]:
chunk_summaries_rut5

['Потому что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу сказать, что я хочу делать пост голосовалку',
 'Что нужно делать тестировщику, которая может улучшить процесс обучения, именно поэтому мы пришли к тебе в бета-тестировании по материалам, которая может улучшить процесс учёбы, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы в поиске бета-тестировщиков, именно поэтому мы сможем выдать дипломы о прохождении курса, которая может улучшить процесс учёбы',
 'Что может быть неплохо, если все участники записывали туда какие-то кейсы рабочие, Инсайты, а тоже отправила заявку без NDA, как только оставила заявку без NDA, как только оставила заяв

In [20]:
chunk_summaries_gpt3

['Да, у меня есть своя колонка о том, как я провёл лето в Турции и что было интересного в моей жизни. О том же, с чего начать, расскажу в скором времени, а пока вот что.\nНо сначала несколько слов о моём новом проекте, где я уже более 2,5 лет и с радостью представляю вам свой авторский курс по аналитике в видеоформате. Этот курс будет не только обучающим, но и познавательным, так как в нем я расскажу о таких вещах, о которых большинство людей, даже в курсе, не знают. \n\nЭтот курс построен по принципу "вопрос-ответ". Каждый день, начиная с понедельника, вы будете задавать мне вопросы, на которые я буду вам отвечать, пока вы',
 'Автор - студент факультета рекламы и маркетинга, в рамках курса "Коммерческая реклама" читает лекцию на тему "Ключевые проблемы коммерческого продвижения".\n\nА кто-нибудь в курсе?\nСможет ли он с помощью этого материала написать статью для журнала, или все-таки нет?\nТворчество и жизнь Оригинал взят у в Творчество в картинках.\n"Пять минут на раздумье"\nhttps:/

### Вывод

 - Попробовал использовать для сумаризации:
     - mT5 - многоязыковую версию T5, предварительно обученную на многоязыковом корпусе Common Crawl (mC4), охватывающем 101 язык;
     - mBART-50	многоязыковую версию BART, предварительно обученную на 50 языках;
     - rut5 - русскоязычную модель на основе модели google/mt5-small;
     - rugpt3large_based_on_gpt2 - архитектура модели основана на GPT-2, но обучение фокусировалось на данных русского языка.
 - mT5 на выхожде не дает осмысленного текста и добавляет специальные токены, такие как, например, 'extra_id_0', не смотря на то, что установил параметр, который должен их убирать skip_special_tokens=True.
 - mBART-50 дала хорошший результат.
 - rut5 дала суммаризацию лучше чем mT5. Но есть многочисленные повторы слов в суммаризованной информации.
 - rugpt3large_based_on_gpt2 - не смотря на мои ожидания, недостаточно хорошо суммаризировала текст.
 - Использовались небольшие модели, но и на них суммаризация на офисном ноутбуке проиходит очень медленно.
 - Возможно, следует использовать для задач суммаризации нейронки через API, если нет ресурсов для локального использования более мощных моделей.